# Project 1: Language Modeling and Fake News Classification

Names: Ikra Monjur, Komukill Loganathan

Netids: im324, kl866

**After you make your own copy, please rename this notebook by clicking on it's name in the upper left corner.** It should be named: 4740_FA20_p1_netid1_netid2

Don't forget to share your newly copied notebook with your partner!

**Reminder: both of you can't work in this notebook at the same time from different computers/browser windows because of sync issues. We even suggest to close the tab with this notebook when you are not working on it so your partner doesn't get sync issues.**

## Introduction
In this project we will build an **n-gram-based language model** for fake news classification. We will also investigate a feature-based **Naive Bayes model**. The task we are faced with is to **decide whether a news article is fake or real**. While some fake articles are so absurd and nonsensical that one can clearly guess they are fake, most fake news is actually quite hard to detect. [Various studies](https://pacscenter.stanford.edu/research/program-on-democracy-and-the-internet/deception-detection-accuracy-for-fake-news-headlines-on-social-media/) have shown that most people can have an error rate up to 50% depending on the theme of the article. 

To help us approach this problem, we will use NLP techniques covered thus far to frame this as a (supervised) binary classification task, where each article will have a label $y \in \{0,1\}$, where *0 indicates a fake article* and *1 indicates a real one*. You will train and validate your two different models and then run them on a test data set with hidden $y$ labels. You will then submit the results on the test data set to Kaggle to participate in our class-wide competition!

The project is divided into six parts:
1. Dataset loading and preprocessing
2. Unsmoothed n-gram language model (LM): build the unsmoothed n-gram language model using our Fake News corpus. 
3. Smoothed n-gram language model: build a smoothed version of the model from part 2.
4. Perplexity: compute perplexity for both the unsmoothed and smoothed model
5. Putting everything together and submitting the first model to Kaggle
6. Naive Bayes: build a feature-based Naive Bayes model to perform the same classification task. Compare the LM with Naive Bayes and identify the pros and cons of each.

**Logistics:** You should work in **groups of 2 students**. Students in the same group will get the same grade. Thus, you should make sure that everyone in your group contributes to the project. 
- **Remember to form groups on BOTH CMS and Gradescope** or not all group members will receive grades. You can use the Teammate Search option on Piazza to find a partner for this project.

**Advice:** Please complete the written parts of this notebook in a clear and informative way. This is where you get to show us that you understand not only what you are doing but also why and how you are doing it. So be clear, organized and concise; avoid vagueness and excess verbiage. Spend time
doing error analysis for the models. This is how you understand the advantages and drawbacks of the systems you build. 
- It's also useful to think about how the theory of n-grams/Naive Bayes bridges with the real world application we are building. Think about what you expect from these models based on your current understanding, and then see if your expectation aligns with empirical results that you'll get. 

## General Guidelines
In this project, we provide a few code snippets or starter points in case you need them. You DO NOT need to follow the structure. 

If you think you have a better idea, go for it. You can ADD, MODIFY, or DELETE any code snippets given to you.

**Let's do this** 🚀

### Dataset

You are given a **News Corpus** on CMS, which consists of roughly the same amount of real and fake news articles.

Real news example:
```
The OpenAI technology, known as GPT-2, is designed to predict the next word given all the previous words it is shown within some text. The language-based model has been trained on a dataset of 8 million web pages.
```

Fake news example:
```
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
```

In the dataset folder you should find 4 files, training and validation splits for both real and fake news.

The project will proceed generally as follows in terms of code development:
1. Write code to train unsmoothed unigram and bigram language models for an arbitrary corpus
2. Implement smoothing and unknown word handling. 
3. Implement the Perplexity calculation. 
4. Using 1, 2 and 3, together with the provided training and validation sets, develop a language-model-based approach for Fake News Classification.
5. Apply your best language-model-based news classifier (from 4) to the
provided test set. Submit the results to the online Kaggle competition. 
6. Use any existing implementation of Naive Bayes (and the provided training and validation sets) to create an additional Naive Bayes fake news classifier. Apply your best NB classifier to the provided test set. Submit the results to the separate Kaggle competition (for NB classifiers). 

We will progress towards these tasks throughout this notebook.

# Part 1: Preprocessing the Dataset
In this part, you are going to do a few things:
* Connect to the google drive where the data set is stored
* Load and read files
* Preprocess the text

------
**Please upload the dataset to each partner's individual Google Drive now.** We suggest using the same folder structure within Google Drive because the notebook is shared among you, so the code to load the data would have to be changed every time if folder structures are different. One folder structure might be: Google Drive/CS 4740/Project 1/Dataset/ or whatever works for you. See our code below for an example of how we load the data from Google Drive.

## 1.1 Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## 1.2 Load and read files
First, let's install [NLTK](https://www.nltk.org/), a very widely package for NLP preprocessing (and other tasks) for Python.

In [ ]:
!pip install -U nltk

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.5)


Then we read and load data.

In [62]:
import os
import io
from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

root_path = os.path.join(os.getcwd(), "drive", "My Drive/CS4740/Project1") # replace based on your Google drive organization
dataset_path = os.path.join(root_path, "real_fake") # same here

with io.open(os.path.join(dataset_path, "trueDataTrain.txt"), encoding='utf8') as real_file:
  real_news = real_file.read()
with io.open(os.path.join(dataset_path, "trueDataValidation.txt"), encoding='utf8') as real_file:
  real_news_validation = real_file.read()
with io.open(os.path.join(dataset_path, "fakeDataTrain.txt"), encoding='utf8') as fake_file:
  fake_news = fake_file.read()
with io.open(os.path.join(dataset_path, "fakeDataValidation.txt"), encoding='utf8') as real_file:
  fake_news_validation = real_file.read()

tokenized_real_news_training = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(real_news)]
tokenized_fake_news_training = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(fake_news)]
tokenized_real_news_validation = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(real_news_validation)]
tokenized_fake_news_validation = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(fake_news_validation)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sanity checks for our real and fake training sets

In [ ]:
tokenized_real_news_training[0]

['london',
 '(',
 'reuters',
 ')',
 '-',
 'britain',
 'is',
 'seeking',
 'to',
 'build',
 'on',
 'the',
 'recent',
 'momentum',
 'in',
 'the',
 'brexit',
 'divorce',
 'talks',
 'with',
 'the',
 'european',
 'union',
 'before',
 'a',
 'summit',
 'next',
 'month',
 ',',
 'a',
 'spokeswoman',
 'for',
 'britain',
 's',
 'department',
 'for',
 'exiting',
 'the',
 'european',
 'union',
 'said',
 'on',
 'tuesday',
 '.']

In [ ]:
tokenized_fake_news_training[0]

['rick',
 'santorum',
 'had',
 'the',
 'impossible',
 'job',
 'of',
 'defending',
 'donald',
 'trump',
 'during',
 'friday',
 's',
 'real',
 'time',
 'and',
 'he',
 'found',
 'himself',
 'humiliatingly',
 'outnumbered.bill',
 'maher',
 'began',
 'by',
 'expressing',
 'astonishment',
 'at',
 'how',
 'the',
 'republican',
 'nominee',
 'has',
 'been',
 'acting',
 'throughout',
 'the',
 'campaign',
 '.']

## 1.3 Data Preprocessing & Preparation

There's a well-known parable in machine learning that 80% of the work is all about data preparation, 10% is supporting infrastructure and 10% is actual modeling. If your "raw" dataset is not preprocessed and prepared in a way to maximize its value, then your model will be more like this: https://xkcd.com/1838/. For this project, modeling is the star of the show for learning purposes, but we still want you to pay attention to the preprocessing stage.

*We've already tokenized and lowercased* the raw data for you. Here are a few extra things you might want to do:

- Think about edge cases. For example, you don't want to accidentally append a period to the last word of a sentence. 
- Watch out for apostrophes and other tricky things like quotations, they cause lots of edge cases. For example, "they're" can be all one token, or two tokens ("they", "'re") or even three tokens ("they", " ' ", "re"). 

Why did we lowercase all tokens? Because the computer will otherwise consider "The" and "the" as two separate words and this will cause problems.

Note that you may use existing
tools just for the purpose of preprocessing. 

Advice: don't get bugged down in the dozens of preprocessing packages and suggestions that you can find on Towards Data Science or Stack Overflow. Start with this [NLTK tutorial](https://lost-contact.mit.edu/afs/cs.pitt.edu/projects/nltk/docs/tutorial/introduction/nochunks.html#:~:text=The%20Natural%20Language%20Toolkit%20(NLTK,tokenization%2C%20tagging%2C%20and%20parsing.) and that should be plenty.

In [63]:
# TODO: preprocessing
#only taking words
import re
from collections import Counter
nltk.download('wordnet')
lemma = nltk.wordnet.WordNetLemmatizer()
snowball = nltk.stem.SnowballStemmer("english")

regex = re.compile("[a-z0-9]+")
def preprocess_uni(tokenized_data):
  uni_dict = Counter()
  clean_tokenized_list = []
  vocab_set = set()
  num_words = 0
  uni_dict["<s>"] = len(tokenized_data)
  uni_dict["</s>"] = len(tokenized_data)
  for tokens in tokenized_data:
    clean_tokenized_list.append("<s>")
    for token in tokens:
      if (regex.match(token)):
        token = lemma.lemmatize(token)
        token = snowball.stem(token)
        # UNKNOWN WORD HANDLING - replace first occurrence of each word with <UNK> token
        if (token not in vocab_set):
          vocab_set.add(token)
          token = "<UNK>"
        uni_dict[token]+=1
        num_words+=1
        clean_tokenized_list.append(token)
    clean_tokenized_list.append("</s>")
      
  return uni_dict, num_words, clean_tokenized_list

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [64]:
uni_real_dict, real_words, tokenized_real_news_training_flat = preprocess_uni(tokenized_real_news_training)
uni_fake_dict, fake_words, tokenized_fake_news_training_flat = preprocess_uni(tokenized_fake_news_training)

In [65]:
print(uni_real_dict["donald"])

7337


In [66]:
print(tokenized_real_news_training_flat[0:50])

['<s>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'the', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'the', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'a', '<UNK>', '<UNK>', 'britain', '<UNK>', '<UNK>', 'for', '<UNK>', 'the', 'european', 'union', '<UNK>', 'on', '<UNK>', '</s>', '<s>', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'we', '<UNK>', '<UNK>', 'to']


In [67]:
def preprocess_bi(tokenized_data):
  bi_dict = Counter()
  for i in range(1, len(tokenized_data)):
    bi_dict[(tokenized_data[i-1],tokenized_data[i]) ] +=1 
  return bi_dict

In [68]:
bi_real_dict = preprocess_bi(tokenized_real_news_training_flat)
bi_fake_dict = preprocess_bi(tokenized_fake_news_training_flat)

In [69]:
print(bi_fake_dict.most_common(10))

[(('</s>', '<s>'), 232436), (('of', 'the'), 37846), (('in', 'the'), 27409), (('<s>', 'the'), 19480), (('to', 'the'), 19395), (('on', 'the'), 13084), (('it', 's'), 12192), (('to', 'be'), 11614), (('for', 'the'), 11267), (('that', 'the'), 10190)]


In [70]:
print(bi_fake_dict[("donald", "trump")])

9410


In [71]:
print(uni_real_dict["shopped"])

0


In [72]:
bi_fake_dict[("<UNK>", "<UNK>")]

9541

In [73]:
def preprocess_test(tokenized_data):
  new_test = []
  for tokens in tokenized_data:
    new_test.append("<s>")
    for token in tokens:
      if (regex.match(token)):
        token = lemma.lemmatize(token)
        token = snowball.stem(token)
        new_test.append(token)
    new_test.append("</s>")
  return new_test

In [74]:
real_valid_lst = preprocess_test(tokenized_real_news_validation)
fake_valid_lst = preprocess_test(tokenized_fake_news_validation)

In [75]:
print(real_valid_lst[:50])

['<s>', 'washington', 'reuter', 'u.s.', 'presid', 'donald', 'trump', 's', 'lawyer', 'on', 'tuesday', 'deni', 'that', 'he', 'or', 'trump', 'collud', 'with', 'russia', 'to', 'interfer', 'in', 'last', 'year', 's', 'presidenti', 'elect', 'and', 'said', 'such', 'charg', 'were', 'meant', 'to', 'discredit', 'trump', 's', 'presid', '</s>', '<s>', 'i', 'emphat', 'state', 'that', 'i', 'had', 'noth', 'to', 'do', 'with']


**Q1.1: Show some observations or statistics from the dataset** (should be quantitative – i.e. most frequent words, most frequent bigram, etc.)

Your answer: We showed the 10 most frequent words and bigrams as well as the 50 least common words and bigrams below:

In [76]:
# TODO: observations/statistics
print("Unigram real dataset most common words")
print(uni_real_dict.most_common(10))
print("Unigram fake dataset most common words")
print(uni_fake_dict.most_common(10))
print("Bigram real dataset most common words")
print(bi_real_dict.most_common(10))
print("Bigram fake dataset most common words")
print(bi_fake_dict.most_common(10))

Unigram real dataset most common words
[('the', 338509), ('<s>', 221017), ('</s>', 221017), ('a', 172492), ('to', 172440), ('of', 143665), ('and', 127474), ('in', 126833), ('on', 76219), ('s', 69970)]
Unigram fake dataset most common words
[('the', 371585), ('<s>', 232437), ('</s>', 232437), ('to', 203028), ('a', 187249), ('of', 165531), ('and', 157537), ('<UNK>', 126577), ('in', 115451), ('that', 105822)]
Bigram real dataset most common words
[(('</s>', '<s>'), 221016), (('<s>', 'the'), 35868), (('of', 'the'), 33510), (('in', 'the'), 28847), (('to', 'the'), 15615), (('said', '</s>'), 15158), (('in', 'a'), 12916), (('on', 'the'), 11710), (('for', 'the'), 10820), (('the', 'unit'), 9961)]
Bigram fake dataset most common words
[(('</s>', '<s>'), 232436), (('of', 'the'), 37846), (('in', 'the'), 27409), (('<s>', 'the'), 19480), (('to', 'the'), 19395), (('on', 'the'), 13084), (('it', 's'), 12192), (('to', 'be'), 11614), (('for', 'the'), 11267), (('that', 'the'), 10190)]


In [77]:
print("Unigram real dataset least common words")
print(uni_real_dict.most_common()[-50:-1])

Unigram real dataset least common words
[('unsupport', 1), ('government-support', 1), ('guillotin', 1), ('olympus', 1), ('1915', 1), ('earpiec', 1), ('london/brussel', 1), ('baes.l', 1), ('12-hour', 1), ('flashmob', 1), ('posloski', 1), ('balance.', 1), ('landau', 1), ('oceania', 1), ('three-prong', 1), ('extraterritori', 1), ('iranian-born', 1), ('easy-go', 1), ('46.3', 1), ('tmsnrt.rs/2ugjcjo', 1), ('gro', 1), ('brundtland', 1), ('24-16', 1), ('hide-and-seek', 1), ('clamber', 1), ('kiev-bas', 1), ('razumkov', 1), ('rania', 1), ('344', 1), ('u.s.-control', 1), ('readership', 1), ('edelman', 1), ('646,000', 1), ('prout', 1), ('programmat', 1), ('2330', 1), ('hanlon', 1), ('headteach', 1), ('whitesupremacist', 1), ('feasible.', 1), ('profits.', 1), ('litman', 1), ('ajala', 1), ('abdelbari', 1), ('url', 1), ('caffo', 1), ('stimson', 1), ('civilians.', 1), ('linden', 1)]


In [78]:
print("Unigram fake dataset least common words")
print(uni_fake_dict.most_common()[-50:-1])

Unigram fake dataset least common words
[('to.although', 1), ('jong-un.even', 1), ('reviewread', 1), ('reinvent', 1), ('election.stev', 1), ('radditz', 1), ('v=eyyqpynhcso', 1), ('tribuneth', 1), ('time.along', 1), ('agency.s', 1), ('joong', 1), ('newspaper.north', 1), ('war.both', 1), ('armistic', 1), ('treaty.for', 1), ('books.bil', 1), ('otherwise.bil', 1), ('act.behind', 1), ('denied.clinton', 1), ('negative.rumor', 1), ('60s.it', 1), ('order.ben', 1), ('dolly.th', 1), ('writes.st', 1), ('picture.featur', 1), ('latest.com', 1), ('ca.her', 1), ('production.ferguson', 1), ('activists.a', 1), ('5th.phelim', 1), ('pic.twitter.com/uycug6mjw1', 1), ('andel', 1), ('lectern', 1), ('resplend', 1), ('diamond/getti', 1), ('indirectly.th', 1), ('pic.twitter.com/degadgnm91', 1), ('conclusive.speci', 1), ('thimon', 1), ('behavior.it', 1), ('105,000', 1), ('homicide-rel', 1), ('pic.twitter.com/3sdkei6b4h', 1), ('places.on', 1), ('goldenshow', 1), ('images/spenc', 1), ('un-new', 1), ('gigabyt', 1)

In [79]:
print("Bigram real dataset least common words")
print(bi_real_dict.most_common()[-50:-1])

Bigram real dataset least common words
[(('atom', 'guardian'), 1), (('guardian', 'surveil'), 1), (('drone', 'final'), 1), (('final', 'secur'), 1), (('drone', 'ha'), 1), (('expand', 'sale'), 1), (('technolog', 'control'), 1), (('1987', 'accord'), 1), (('it', 'categor'), 1), (('categor', 'drone'), 1), (('drone', 'with'), 1), (('rang', 'greater'), 1), (('mile', '300'), 1), (('payload', 'abov'), 1), (('abov', '1,100'), 1), (('1,100', 'pound'), 1), (('pound', '500'), 1), (('kg', 'a'), 1), (('missil', 'requir'), 1), (('requir', 'extrem'), 1), (('intern', 'stamp'), 1), (('relax', 'u.s.'), 1), (('mtcr', 'renegoti'), 1), (('the', 'missile-control'), 1), (('missile-control', 'group'), 1), (('dublin', 'next'), 1), (('paper', 'propos'), 1), (('creat', 'be'), 1), (('lenient', 'than'), 1), (('mtcr', 'wa'), 1), (('ha', 'nato'), 1), (('could', 'resist'), 1), (('an', 'mtcr'), 1), (('mtcr', 'signatori'), 1), (('signatori', 'ha'), 1), (('iraq', 'saudi'), 1), (('pas', 'u.s.'), 1), (('regulatori', 'muster'

In [80]:
print("Bigram fake dataset least common words")
print(bi_fake_dict.most_common()[-50:-1])

Bigram fake dataset least common words
[(('number', 'address'), 1), (('like', 'height'), 1), (('height', 'weight'), 1), (('forth', 'graham'), 1), (('contain', 'address'), 1), (('real', 'gps'), 1), (('tech', 'news'), 1), (('discov', 'british'), 1), (('and', 'vatican'), 1), (('vatican', 'staff'), 1), (('staff', 'among'), 1), (('instanc', 'sever'), 1), (('at', 'whitehouse.gov'), 1), (('whitehouse.gov', 'wherea'), 1), (('wherea', 'white'), 1), (('email', 'communications.th'), 1), (('communications.th', 'hacker'), 1), (('wa', 'avid'), 1), (('embarrass', 'now'), 1), (('hill', 'eric'), 1), (('communiti', 'veteran'), 1), (('fact', 'everyth'), 1), (('theorist', 'donald'), 1), (('host', 'said.er'), 1), (('said.er', 'said'), 1), (('the', 'ibd'), 1), (('ibd', 'poll'), 1), (('rasmussen', 'we'), 1), (('state', 'haven'), 1), (('presid', 'romney'), 1), (('romney', 'how'), 1), (('fight', 'from'), 1), (('behind', 'right'), 1), (('listen', 'mayb'), 1), (('love', 'out'), 1), (('eric', 'continu'), 1), (('h

**Please answer the following question**:

**Q1.2: What did you do in your preprocessing part?**

Example answer format:

A: We tokenized and lowercased all the words.

Your answer: We first removed all punctuations using regular expressions. Then we used nltk libraries (WordNet Lemmatizer and Snowball Stemming) to lemmatize and stem real and fake training data because we wanted to remove the possibility of different tenses, verb agreements and other grammatical variations from skewing the model. 

We added sentence beginning and end tokens. We also handled unknown words by changing the first occurrence of each word to the \<UNK> token.

We also built unigram and bigram count dictionaries respectively.

We added a method to preprocess the test and validation data which does everything that is stated here.

# Part 2: Compute Unsmoothed Language Models.

To start, you will write a program that computes unsmoothed unigram and bigram probabilities. You should consider real and fake news as separate corpora and
generate a separate language model for each set of news.
We have already loaded the data and (partially) preprocessed it and you probably did some of your own preprocessing. 

Note that you were allowed to use existing
tools for the purpose of preprocessing, but you must write the code for gathering n-gram counts and computing n-gram probabilities yourself. 

For example, consider the
simple corpus consisting of the sole sentence:


> the students liked the assignment

Part of what your program would compute for a unigram and bigram model, for example,
would be the following:


> $P("the") = 0.4; P("liked") = 0.2; P("the"|"liked") = 1.0; P("students"|"the") = 0.5$

Remember to add a symbol to mark the beginning of sentence. See Sept. 7th lecture, p25-28 for an example.




**Advice**: jupyter notebooks (including colab) can be a double-edged sword. It's amazing and liberating to just start writing code and run it by simply running a cell. However, it gets messy very quickly. So, once you're done prototyping, we highly recommend using functions (classes may be unnecessary but go for it if you want) to make things cleaner and easier to debug.

## 2.1 Unsmoothed Uni-gram Model.

In this part of the project, you are trying to compute the probabilities for a unigram model. You might want to take in a list of words, and return the probabilities for each
occurence. Think of an efficient data structure to use here given what ratio of reads and puts you expect.

Please look at the example above and consider how we get the probabilities.

Below is a starter point you can go from, but you DO NOT need to stick it. Feel free to use your own design.

In [81]:
"""
Function [unsmoothed_unigram] computes the probabilities for a unigram model
lst: a list of words in test data
dic: unigram dictionary of the model
total_words: the total number of words in the training corpus of the model
Return: dictionary that stores the probability of each unigram in [lst]
"""
def unsmoothed_unigram(lst, dic, total_words):
  word_probs = Counter()
  for word in lst:
    word = lemma.lemmatize(word)
    word = snowball.stem(word)
    word_probs[word] = dic[word]/total_words
  return word_probs

In [82]:
unsmoothed_unigram(["i", "love", "donald", "trump", "love", "you"], uni_fake_dict, fake_words)

Counter({'donald': 0.0017429607516461058,
         'i': 0.004387280389375602,
         'love': 0.000338098931893294,
         'trump': 0.007512086500717513,
         'you': 0.004035743275834118})

## 2.2 Unsmoothed Bi-gram Model.

In this part of the project, you are trying to compute the probabilities for a bigram model. You can approach this with similar methods as above.

Remember the definition:
$p(w_n\mid w_{n-1})=\frac{C(w_{n-1}w_n)}{C(w_{n-1})}$ this means you might want to store two things (count of $w_{n-1}$ and count of $w_{n-1}w_n$).

In [83]:
# TODO: Add code for bigram probability calculation. 
"""
Function [unsmoothed_bigram] computes the probabilities for a bigram model
lst: a list of words in test data
unigram_dict: unigram dictionary of the model
bigram_dict: bigram dictionary of the model
Return: dictionary that stores the probability of each bigram in [lst]
"""
def unsmoothed_bigram(lst, bigram_dict, unigram_dict):
  word_probs = Counter()
  for i in range(1, len(lst)):
    prev_word = snowball.stem(lemma.lemmatize(lst[i-1]))
    curr_word = snowball.stem(lemma.lemmatize(lst[i]))
    word_probs[(prev_word, curr_word)] = bigram_dict[(prev_word, curr_word)]/unigram_dict[prev_word]
  return word_probs

In [84]:
unsmoothed_bigram(["i", "love", "donald", "trump", "love", "you"], bi_fake_dict, uni_fake_dict)

Counter({('donald', 'trump'): 0.7718175853018373,
         ('i', 'love'): 0.007592296914203787,
         ('love', 'donald'): 0.004228329809725159,
         ('love', 'you'): 0.026215644820295984,
         ('trump', 'love'): 0.0007612232858203133})

**Please answer the following question**:

**Q2: What data structure are you using to store probabilities for unigrams and bigrams? Why did you select this data structure?**

Your answer: We use Counter() which is a type of dictionary. We chose this because it has a constant lookup and put time. We didn't want to use a list because (1) operations on list are expensive, (2) the probabilities of repeating unigrams and bigrams are the same in this model and we don't need duplicates.

# Part 3: Smoothed Language Model
In this part, you will need to implement **at least one** smoothing method and **at least one** method to handle unknown words in the test data. You can choose any method(s) that you want for each. You should make clear
**what method(s)** were selected and **why**, providing a description for any non-standard approach (e.g., an approach that was not covered in class or in the readings). 

You should use the
provided validation sets to experiment with different smoothing/unknown word handling
methods if you wish to see which one is more effective for this task. (We will cover this in Part 4).

## 3.1 Unknown Words Handling

**Please answer the following questions:**

**Q3.1: How are you going to handle unknown words? What parameters might be needed? Do you need a method to determine the value?**

Your answer: We handled unknown words in the preprocessing section. We replaced the first occurrence of each word by \<UNK> token. We don't need any parameters to do this because we simply replaced the first occurrences of the words in our training corpus as we pre-processed them. The first occurrences were tracked by using a vocab set. As we pre-processed the words, we checked if they were in our vocab set. If they were not, we made that word to be \<UNK> and added them to the vocab set.


In [85]:
# TODO: Add your unknown word handling code 

# UNKNOWN WORD HANDLING done in preprocessing in preprocess_unigram().
# Pre-processed input token lists (replacing first occurrences of each word in the vocab) 
# are used to build bigrams as well

## 3.2 Smoothing

In this part of project, we are going to compute the probabilities for unigram and bigram models after smoothing.
There are several smoothing methods you can start with:
* add-k
* Kneser-Ney
* Good-Turing
* ...

You need to compute for both unigram and bigram models.

Below is a starter point using add-k smoothing. As always, you DO NOT need to follow it; you DO NOT need to use add-k smoothing if you do not want to. You can pick ANY smoothing method you want.

In [86]:
"""
Function [add_k_unigram] does add-k smoothing to unigram model.
lst: a list of words in test data
dic: a dictionary of unigrams. key: word, val: frequency
k: parameter k for smoothing
total_words: total words in the training corpus of model
Return: a dictionary of probabilities after smoothing
"""
def add_k_unigram(lst, dic, k, total_words):
  word_probs = Counter()
  for word in lst:
    if dic[word] == 0:
      word = "<UNK>"
    word_probs[word] = (dic[word] + k) / (total_words + k * len(dic))
  return word_probs


In [87]:
"""
Function [add_k_bigram] does add-k smoothing to bigram model.
lst: a list of words in test data
uni_dic: a dictionary of unigrams. key: word, val: frequency
bi_dic: a dictionary of bigrams. key: word-pairs, val: frequency
k: parameter k for smoothing
Return: a dictionary of probabilities after smoothing
"""
def add_k_bigram(lst, uni_dic, bi_dic, k):
  word_probs = Counter()
  for i in range(1, len(lst)):
    prev_word = lst[i-1]
    if uni_dic[prev_word] == 0:
      prev_word = "<UNK>"
    curr_word = lst[i]
    if uni_dic[curr_word] == 0:
      curr_word = "<UNK>"
    pair = (prev_word, curr_word)
    word_probs[pair] = (bi_dic[pair] + k) / (uni_dic[pair[0]] + k * len(uni_dic))
  return word_probs

**Please answer the following question:**

**Q3.2: Which smoothing method did you choose? Are there any parameters, if so how are you planning to pick the value? If you choose to implement more than 1 method (not a requirement), please state each of them. Providing a description for any non-standard approach, e.g., an approach that was not covered in class or in the readings**

Your answer: We use add-k-smoothing. For both unigram and bigram models, we need k as a parameter. We think a smaller value of k is better because a smaller k value does not affect the overall probability as much and therefore, preserves the accuracy of the model. The value is chosen after trying out multiple values of k until we found an appropriate k value.

# Part 4: Perplexity
At this point, we have developed several language models: unigram vs bigram, unsmoothed vs smoothed. We now want to compare all the models. 

Implement code to compute the perplexity of a **“development set.”** (“Development set”
is just another way to refer to the validation set—part of a dataset that is distinct from
the training portion and the test portion.) Compute and report the perplexity of each
of the language models (one trained on true news and fake news) on
the development corpora. Compute perplexity as follows:
\begin{align*}
PP &= \left(\prod_i^N\frac{1}{P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)}\right)^{\frac{1}{N}}\\
&=\exp \frac{1}{N}\sum_{i}^N-\log P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)
\end{align*}
where $N$ is the total number of tokens in the test corpus and $P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)$
is the n-gram probability of your model. Under the second definition above, perplexity
is a function of the average (per-word) log probability: use this to avoid numerical
computation errors.

Please complete the following tasks and report what you have observed. Remember, lower perplexity means better model.

## Task 1: Compute perplexity for smoothed unigram and smoothed bigram. 
*Note: If you choose more than one smoothing method, pick one of them to compute. If you need to try different values of parameters, you can try them out here.*


In [88]:
# TODO: compute perplexity for one smoothing method on unigram, and one smoothing method on bigram.
import math

def perplexity_uni(k, lst, dic, words):
  prob_dict = add_k_unigram(lst, dic, k, words)
  acc = 0
  for word in lst:
    if word not in dic:
      word = "<UNK>"
    acc += -1*math.log(prob_dict[word])
  pp = math.exp((1/len(lst))*acc)
  return pp


def perplexity_bi(k, lst, uni_dic, bi_dic):
  prob_dict = add_k_bigram(lst, uni_dic, bi_dic, k)
  acc = 0
  for i in range(1, len(lst)):
    prev_word = lst[i-1]
    if prev_word not in uni_dic:
      prev_word = "<UNK>"
    curr_word = lst[i]
    if curr_word not in uni_dic:
      curr_word = "<UNK>"
    pair = (prev_word, curr_word)
    acc += -1*math.log(prob_dict[pair])
  pp = math.exp((1/len(lst))*acc)
  return pp


In [89]:
perplexity_uni(0.001, fake_valid_lst, uni_fake_dict, fake_words)

793.7791905377273

In [90]:
perplexity_uni(1, fake_valid_lst, uni_fake_dict, fake_words)

795.4321731851157

In [97]:
perplexity_uni(2, fake_valid_lst, uni_fake_dict, fake_words)

798.2045356650539

In [91]:
perplexity_uni(0.001, real_valid_lst, uni_real_dict, real_words)

722.7353722371856

In [92]:
perplexity_uni(1, real_valid_lst, uni_real_dict, real_words)

723.0357853192074

In [98]:
perplexity_uni(2, real_valid_lst, uni_real_dict, real_words)

724.0934673685134

In [93]:
perplexity_bi(0.001, real_valid_lst, uni_real_dict, bi_real_dict)

195.26234500504205

In [94]:
perplexity_bi(1, real_valid_lst, uni_real_dict, bi_real_dict)

755.5007142743374

In [99]:
perplexity_bi(2, real_valid_lst, uni_real_dict, bi_real_dict)

755.5007142743374

In [95]:
perplexity_bi(0.001, fake_valid_lst, uni_fake_dict, bi_fake_dict)

221.68907551489798

In [96]:
perplexity_bi(1, fake_valid_lst, uni_fake_dict, bi_fake_dict)

1227.778785515183

In [100]:
perplexity_bi(2, fake_valid_lst, uni_fake_dict, bi_fake_dict)

1830.8423206588445

**Q4.1: Why do we need to compute perplexity after smoothing?**

Your answer: We need to compute perplexity after smoothing to ensure that our smoothing method and the value of k (the parameter) does not make our model very uncertain. We want to use perplexity to choose the best smoothing parameter, which is k in our approach, to ensure a better prediction of the test data. A good k value is one that gives a lower perplexity value.

**Q4.2: Did you choose any values for parameters?**

Your answer: At first, we tried with k=1, as it is the standard k for most cases, for all of our models and calculated the perplexity. We then tried making k larger and smaller as showsn in the above cells. We realized that lowering k value decreased perplexity. With that, we determined that k=0.001 will be better for all of our models. Making k lower than 0.001 barely made a difference in perplexity for all the models. So we choose to use k=0.001 for our models.

## Task 2: Compute perplexity for other smoothing methods (optional). 
*Note: If you only pick one smoothing method, you can omit this task. If you need to try different values of parameters, you can try them out here.*

In [ ]:
# TODO: compute perplexity for your rest of smoothing method.

**Q4.3: If your smoothing method needs to pick a parameter, what is the value of your parameter?**

Your answer:

**Q4.4: Which smoothing method is the best among your choices?**

Your answer:

# Part 5: Putting Everything Together and Submitting to Kaggle
Combining all the previous parts together, we have developed a bunch of language models. Before we proceed to the next step, let's check a few things (no need to answer):
* Did you train your model only on training set?
* Did you validate your model only on validation/development set?
* Did you determine all your parameters?

Finally, please answer:

**Q5: What is your choice of language model, and why?**

Your answer: We chose bigram model with add-k-smoothing, where k=0.001, because this model gives us the lowest perplexity on the validation data. A lower perplexity means less uncertainty and the model predicts the validation set better. We determined all of our parameters as stated here after checking with wide choices of parameters.



In [ ]:
# TODO: anything that helps you answer/check the above points.

## Part 5.1: First Model Submission to Kaggle

Now we need to apply our model to testing data. What you need to do:
* Takes the test data as input, and generates an output of your prediction based on your chosen language model
* Your output file should be ONLY your predictions
* Submit to Kaggle

You should use your trained model to predict labels for all the news in `TestData.txt`. Output your predictions to a **csv** file and submit it to kaggle. Each line should contain the id of the test news and its corresponding prediction (in total 4489 lines). In other words, your output should look like (**including the header**):
```
Id,Prediction
0,0
1,0
2,1
3,0
...
4488,1
```
Note that you should add the header `Id,Prediction` and there is no space in the output. The Id starts from 0 (not 1).

Use this kaggle [link](https://www.kaggle.com/t/a032d54898b24b4bb2f29e7165ac5fdf) to submit your output. Your team name should be the concatenation of your netids, **exactly in the same order as this notebook is named**. For example, if notebook is 4740_FA20_p1_jb123_cj456, then Kaggle group should be jb123_cj456.


In [ ]:
# TODO: Add code to generate the Kaggle output file and submit the output file to Kaggle
with io.open(os.path.join(dataset_path, "TestData.txt"), encoding='utf8') as test_file:
  test_data = test_file.read()
test_lst = test_data.split('\n')[:-1]

In [ ]:
"""
Function [calc_prob] calculates the probability of the test_data given a model.
preprocess_test: tokenized and preprocessed list of test data
uni_dic: dictionary of unigram model
bi_dic: dictionary of bigram model
Returns probability (in log base) of the test data given a model
"""
def calc_prob(preprocess_test, uni_dic, bi_dic):
  prob_dict = add_k_bigram(preprocess_test, uni_dic, bi_dic, 0.01)
  prob = 0
  for i in range(1, len(preprocess_test)):
    prev_word = preprocess_test[i-1]
    if uni_dic[prev_word] == 0:
      prev_word = "<UNK>"
    curr_word = preprocess_test[i]
    if uni_dic[curr_word] == 0:
      curr_word = "<UNK>"
    pair = (prev_word, curr_word)
    prob += math.log(prob_dict[pair])
  return prob

In [104]:
import csv

with open(os.path.join(dataset_path, "predictions.csv"), 'w') as csvfile:
  filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
  filewriter.writerow(['Id', 'Prediction'])
  for i in range(len(test_lst)):
    tokenized_test_data = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(test_lst[i])]
    preprocess_test_data = preprocess_test(tokenized_test_data)
    real_prob = calc_prob(preprocess_test_data, uni_real_dict, bi_real_dict)
    fake_prob = calc_prob(preprocess_test_data, uni_fake_dict, bi_fake_dict)
    pred = 0
    if (real_prob > fake_prob):
      pred = 1
    filewriter.writerow([i, pred])

# Part 6: Naive Bayes

The Naive Bayes classification method is based on Bayes Rule. Suppose we have a news article *d* and its label *c* (either 0 or 1).
\begin{align*}
P(c|d)=\frac{P(d|c)P(c)}{P(d)}
\end{align*}
Likelihood: $P(d|c)$. In real/fake corpus, how likely *d* would appear.

Prior: $P(c)$. The probability of real/fake news in general.

Posterior: $P(c|d)$. Given *d*, how likely is it that it is real/fake.

Goal: $\underset{c\in \{0,1\}}{\operatorname{argmax}} P(c|d)$, which is equivalent to $\underset{c\in \{0,1\}}{\operatorname{argmax}} P(d|c)P(c)$.

The equivalence holds because $P(d)$ is the same for any $c$. Thus the denominator can be dropped.

Denote $d=\{x_1, x_2, ..., x_n\}$ where $x_i$'s are words in the news *d* (sometimes called features). Unlike n-gram language modelling, we make the multinomial Naive Bayes independence assumption here, where we assume positions of words do not matter. Formally, 
\begin{align*}
&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(d|c)P(c)\\
=&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(x_1, ..., x_n|c)P(c)\\
=&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(x_1|c)P(x_2|c)...P(x_n|c)
\end{align*}

Now we only need to collect the occurences of each word for the classification. This is often called a **bag of words** feature. 

For instance, in the sentence `All for one and one for all .`, the bag of words feature would be `{"all": 2, "for": 2, "one": 2, "and": 1, ".": 1}`. Essentially, the bag of words feature is a dictionary which maps the word to its occurences. We can see that the order is not considered here.

Now, your goal is to implement the Multinomial Naive Bayes. You can use existing codes or Python packages, and adapt them to our news classification task.

You might find the following packages/functions useful:

* nltk.word_tokenize(), nltk.word_tokenize()
* nltk.classify.naivebayes()
* sklearn.feature_extraction.text
* sklearn.naive_bayes.MultinomialNB()

**Please answer the following question(s).**

**Q6: Comparing Multinomial Naive Bayes with the unigram language model, which one do you expect to perform better? Why?**

A: Our initial intuition is that Multinomial Naive Bayes with bag of words feature that we use here and unigram will have about the same level of performance because both models calculate the probability of a sequence of words independent of the position of a word occurrence in a sequence. 

However, we tried running a unigram model on test data along with Multinomial Naive Bayes. We got about 77% accuracy for unigram model whereas Multinomial Naive Bayes gave us a 95% accuracy. So, Multinomial Naive Bayes performs better than unigram language model. This might be because Multinomial Naive Bayes handles unknown words better than the way we handle it in unigram.

## 6.1 Implementation

In [ ]:
# TODO: Naive Bayes implementation 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

#creating BoW
count_vect = CountVectorizer()
corpus = [real_news, fake_news]
X = count_vect.fit_transform(corpus)
Y = [1,0]
model = MultinomialNB()
model.fit(X, Y)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import classification_report

validation_corpus = [real_news_validation, fake_news_validation]
X_valid = count_vect.transform(validation_corpus)
y_preds = model.predict(X_valid)
Y_valid = [1,0]
print(classification_report(Y_valid, y_preds))
print(y_preds)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

[1 0]


## 6.2 Putting Everything Together and Submitting to Kaggle

You should use your trained model to predict labels for all the news in `TestData.txt`. Output your predictions to a **csv** file and submit it to kaggle. The format should follow Part 6 as well.

Use this Kaggle [link](https://www.kaggle.com/t/e8e4f7ee507843d1902c168529d705c8) to submit your output.

In [ ]:
# TODO: Code for predicting the test labels and generating the output file. Then submit the output file to Kaggle

with open(os.path.join(dataset_path, "NBpreds.csv"), 'w') as csvfile:
  filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
  filewriter.writerow(['Id', 'Prediction'])
  X_test = count_vect.transform(test_lst)
  Y_preds = model.predict(X_test) 
  for i in range(len(Y_preds)):
    filewriter.writerow([i, Y_preds[i]])

# Work Distribution

**Please briefly describe how you divided the work.**

Your answer: We met up and worked on the project together by pair-programming, i.e. taking turns typing and collaboratively discussing approaches.


# Project Feedback [1 point]
It goes without saying that this is an unprecedented time. We on the course staff are trying our best to adapt our teaching, projects and everything else in the class to this new modality. We would immenselly appreciate it if you could provide us feedback (it's a super short form!!) on this project and **it's worth 1 point of your project grade**

Link to the feedback form: https://forms.gle/xfonP9rJk9Uk7zaSA 

We will use this feedback to improve both **upcoming projects** and projects for next year. 

Thank you so much!

# Submitting the Notebook

1. Go to File (upper left corner) -> Download .ipynb -> submit this downloaded file to cms
2. Run the first code block
3. Replace our placeholder for your correct Google Drive directory structure in the 2nd code block below. Run the code block
4. Put the name of this notebook into our placeholder in the 3rd code block. Run the code block
5. Then go to the folder icon on the very left panel, under the orange CO logo. Click on the folder and wait for a PDF version of your notebook to appear. Might take a few minutes.
6. Download the pdf version and submit to Gradescope

In [ ]:
%%capture
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [ ]:
%%capture
# the red text is a placeholder! Change it to your directory structure!
!cp 'drive/My Drive/Colab Notebooks/4740_FA20_p1_aak85_jb123.ipynb' ./ 

In [ ]:
# the red text is a placeholder! Change it to the name of this notebook!
!jupyter nbconvert --to PDF "4740_FA20_p1_aak85_jb123.ipynb"

# You are done! ✅